In [6]:
#Takes the input paragraph and splits it into a list of sentences
from sentence_splitter import SentenceSplitter, split_text_into_sentences
 
splitter = SentenceSplitter(language='en')

#Create a list of sentences from a text file

def create_list(text):
    with open(text, 'r',encoding='utf-8') as file:
        data = file.read().replace('\n', '')
        #print(data)
        sentence_list = splitter.split(data)
    return sentence_list

In [7]:
import pandas as pd
import datasets
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os

In [9]:
config = LongformerConfig()
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', max_length = 1024)
config = LongformerConfig.from_json_file('E:/Research/Plagiarism/Longformer_Epoch-1/config.json')
model = LongformerForSequenceClassification.from_pretrained('E:/Research/Plagiarism/Longformer_Epoch-1/pytorch_model.bin',config=config)

In [50]:
def create_prediction(sentence_list):
    final_output = []
    for i in range(20):
        tokenized_sentence = tokenizer.encode(sentence_list[i])
        input_ids = torch.tensor([tokenized_sentence])
        with torch.no_grad():
            output = model(input_ids)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=1)
        print(int(label_indices))
        final_output.append(int(label_indices))
    count_1 = final_output.count(1)
    count_0 = final_output.count(0)
    pct_plagiarism = (count_1/len(final_output))*100
    
    return str(pct_plagiarism)+'%'